# Обработка сущностей из Викиданных и создание словарей

In [6]:
def read_tsv(file):
    with open(file, "r") as f:
        f = f.readlines()
    data = []
    for line in f:
        line = line.strip('\n')
        q, entity, aliases = line.split('\t')
        data.append([q, entity, aliases])
    return data

Читаем tsv-файл, в котором собраны сущности из русскоязычной версии Викиданных, их уникальные идентификаторы Q и другие возможные наименования. 

In [7]:
%%time
data = read_tsv("/Users/anyway/Desktop/all-entities-bigfile.tsv")

CPU times: user 40.6 s, sys: 2min 14s, total: 2min 55s
Wall time: 6min 10s


In [8]:
len(data)

5837666

In [10]:
data[1226]

['Q11349', 'туманность Орёл', 'M 16|NGC 6611']

In [2]:
%%time
import re


#чистим Викиданные
def clean_wikidata(data):
    russian = "[А-Яа-я]+"
    prefixes = ("Категория:", "Шаблон:", "Приложение:", "ВЭ /", "БЭАН /", "ЭСБЕ /", "РБС /", "БЭЮ /", "ЭЛ /", 
                "МЭСБЕ /", "ЕЭБЕ /", "ТСД /", "ТСД2 /", "ПБЭ /", "БСЭ1 /", "НЭС /", "ББСРП /", "РСКД /", "ТСД3 /") 
    data = [i for i in data if len(i[1]) > 2 
            and re.search(russian, i[1]) 
            and not i[1].startswith(prefixes)]
    for indx, line in enumerate(data):
        aliases = line[2].split('|')
        data[indx][2] = " | ".join([i for i in aliases if re.search(russian, i)])
    with open('clean-russian-wikidata.tsv', 'w') as f:
        for i in data:
            f.write("\t".join(i) + '\n')
    return data

clean_data = clean_wikidata(data)    
len(clean_data)

CPU times: user 19.2 s, sys: 4.5 s, total: 23.7 s
Wall time: 24.9 s


In [79]:
print(len(clean_data))
clean_data[:100]

2234352


[['Q31', 'Бельгия', 'Королевство Бельгия'],
 ['Q8', 'счастье', ''],
 ['Q23', 'Джордж Вашингтон', 'Вашингтон, Джордж'],
 ['Q24', 'Джек Бауэр', ''],
 ['Q42', 'Дуглас Адамс', 'Адамс, Дуглас'],
 ['Q1868', 'Поль Отле', 'Отле, Поль'],
 ['Q2013', 'Викиданные', 'Викидата'],
 ['Q45', 'Португалия', 'Португальская Республика'],
 ['Q51', 'Антарктида', ''],
 ['Q58', 'пенис', 'детородный орган | половой член | член'],
 ['Q68', 'компьютер', ''],
 ['Q75', 'Интернет', ''],
 ['Q125', 'ноябрь', 'ноября'],
 ['Q140', 'лев', ''],
 ['Q144', 'собака', 'собака обыкновенная'],
 ['Q147', 'котёнок', 'Котёнок | Котенок'],
 ['Q148', 'Китайская Народная Республика', 'КНР | Китай | Поднебесная'],
 ['Q155', 'Бразилия', 'Федеративная Республика Бразилия'],
 ['Q163', 'Йоркшир', ''],
 ['Q177', 'пицца', ''],
 ['Q178', 'макаронные изделия', 'паста | макароны'],
 ['Q183', 'Германия', 'Федеративная Республика Германия | ФРГ | Дойчланд'],
 ['Q207', 'Джордж Уокер Буш', 'Джордж Буш - младший | Джордж Буш-младший'],
 ['Q210', 'п

In [92]:
%%time

from pymorphy2 import MorphAnalyzer
from string import punctuation

punct = punctuation+'«»—…“”*–'
morph = MorphAnalyzer()

#лемматизируем Викиданные и почистим от лишней полнктуации
def normalize(data):
    norm_data = []
    for line in data:
        entity = [word.strip(punct) for word in line[1].lower().split()]
        entity = [morph.parse(word)[0].normal_form for word in entity if word]
        aliases = line[2].lower()
        if aliases != '':
            if ' | ' in aliases:
                aliases = [alias.split() for alias in aliases.split(" | ")]
                aliases = [[morph.parse(word.strip(punct))[0].normal_form for word in alias if word] 
                           for alias in aliases]
                norm_data.append([line[0], [entity] + aliases])
            else:
                aliases = [word.strip(punct) for word in aliases.split()]
                aliases = [morph.parse(word)[0].normal_form for word in aliases if word]
                norm_data.append([line[0], [entity] + [aliases]])     
        else:
            norm_data.append([line[0], entity])
    return norm_data

norm_data = normalize(clean_data[:100])
#заметила, что нужно еще добавить очистку от диакритиков

CPU times: user 154 ms, sys: 50 ms, total: 204 ms
Wall time: 330 ms


In [12]:
from string import punctuation

punct = punctuation+'«»—…“”*–'
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~«»—…“”*–


In [93]:
norm_data

[['Q31', [['бельгия'], ['королевство', 'бельгия']]],
 ['Q8', ['счастие']],
 ['Q23', [['джордж', 'вашингтон'], ['вашингтон', 'джордж']]],
 ['Q24', ['джек', 'бауэр']],
 ['Q42', [['дуглас', 'адамс'], ['адамс', 'дуглас']]],
 ['Q1868', [['поль', 'отля'], ['отля', 'поль']]],
 ['Q2013', [['викидать'], ['викидата']]],
 ['Q45', [['португалия'], ['португальский', 'республика']]],
 ['Q51', ['антарктида']],
 ['Q58', [['пенис'], ['детородный', 'орган'], ['половый', 'член'], ['член']]],
 ['Q68', ['компьютер']],
 ['Q75', ['интернет']],
 ['Q125', [['ноябрь'], ['ноябрь']]],
 ['Q140', ['левый']],
 ['Q144', [['собака'], ['собака', 'обыкновенный']]],
 ['Q147', [['котёнок'], ['котёнок'], ['котёнок']]],
 ['Q148',
  [['китайский', 'народный', 'республика'],
   ['кнр'],
   ['китай'],
   ['поднебесный']]],
 ['Q155', [['бразилия'], ['федеративный', 'республика', 'бразилия']]],
 ['Q163', ['йоркшир']],
 ['Q177', ['пицца']],
 ['Q178', [['макаронный', 'изделие'], ['паста'], ['макароны']]],
 ['Q183',
  [['германия']

Данный список списков, в котором вложеннный список - это номер Q и список списков возможных наименований сущностей, удобен нам для создания системы словарей. 
Теперь нормализуем все данные:

In [94]:
%%time
norm_data = normalize(clean_data)

CPU times: user 43min 3s, sys: 1min 56s, total: 44min 59s
Wall time: 47min 19s


In [95]:
norm_data[-100:]

[['Q61750033', ['ипполит', 'д’эст']],
 ['Q61750293', ['афанасий', 'череповецкий']],
 ['Q61750550', ['нецианин']],
 ['Q61750774', ['лукас', 'альварес']],
 ['Q61750983', ['жилетта', 'де', 'тренела']],
 ['Q61750985', ['адель', 'де', 'маринья']],
 ['Q61750996', ['бернар', 'iii', 'де', 'сен-валери']],
 ['Q61751027', ['афанасий', 'иван', 'милославский']],
 ['Q61751238', ['свердлов', 'юрий']],
 ['Q61754031', ['галаксиј']],
 ['Q61754323', ['джулия', 'коук']],
 ['Q61754326', ['уолтер', 'плиделла-бовери']],
 ['Q61754328', ['фицгерберт', 'райт']],
 ['Q61754377', ['элджернон', 'фрэнсис', 'холфорд', 'фергюсон']],
 ['Q61754382', ['джон', 'стифенсон', 'фергюсон']],
 ['Q61754407', ['nn', 'фёдор']],
 ['Q61754415', ['михаил', 'давыд', 'моложский']],
 ['Q61755109',
  [['никон', 'валерий', 'владимир'], ['валерий', 'владимир', 'никон']]],
 ['Q61759346', ['микаэль', 'конда']],
 ['Q61762916',
  ['список',
   'город',
   'стать',
   'столица',
   'государство',
   'после',
   '1960',
   'год']],
 ['Q61763420'

## Теперь о том, как будут создаваться словари. 

Допустим, у нас будет всего 7 словарей: 
1. Словарь униграмм, где ключом будет униграмма, а значением - номер Q. 
2. Словарь биграмм, где ключом будет биграмма, а значением - номер Q. 
3. Словарь триграмм. 
4. Словарь тетраграмм. 
5. Словарь пентаграмм. 
6. На всякий случай: Общий обратный словарь, где ключ - это номер сущности Q, а значения - это все возможные наименования этой сущности в нашей системе словарей. Возможно, он пригодится (например, для контроля того, как нарезаются наши сущности). 
7. На всякий случай: Словарь дат, где ключ - это сущность, в которой встречается слово "год" и 4 цифры, а значение - это номер Q. Это будет дополнительный словарь, он может дублировать часть ключей словарей n-грамм. 


Наша функция должна проходиться по чистому нормализованному датасету Викиданных и делать следующее:

1. Создать общий обратный словарь, где номер Q - ключ, значения пока оставим пустыми. 
2. Проверять длину каждого наименования сущности в каждой строке данных. 
- если это сущность длины 1, сразу кладем ее в словарь униграмм. Если такая сущность в словаре уже есть, прибавляем новый номер Q к сущности. То есть, одной сущности в словаре может соответствовать несколько номеров Q. 
- если это сущность длины 2, кладем ее в словарь биграмм, потом меняем элементы местами и снова кладем ее в словарь биграмм. Опять же добавляем номера Q, если вдруг ключи повторяются (на последующих этапах то же самое).  
- если это сущность длины 3, кладем ее в словарь триграмм. Также комбинаторно меняем слова сущности местами и кладем в словарь триграмм с другим ключом. А еще комбинаторно выбираем биграммы из триграмм и кладем в словарь биграмм. Это может быть полезно в случае имен собственных. 
- если это сущность длины 4, кладем ее в словарь тетраграмм, а также ее комбинаторные варианты. Также разбиваем (уже не комбинаторно) сущность на триграммы по порядку (1-2-3 слово, 2-3-4 слово) и кладем эти триграммы в словарь триграмм. 
- если это сущность длины 5, кладем ее в словарь пентаграмм, а также ее комбинаторные варианты. Также разбиваем сущность по порядку на триграммы и тетраграммы и кладем в соответствующие словари. 
- если в сущности есть слово "год" и 4 цифры, кладем ее целиком в качестве ключа в словарь дат. 
3. После того, как мы положили что-то в один из словарей, сразу добавляем это наименование сущности (или ее часть) в обратный словарь. 
4. Функция должна вернуть 7 словарей соответственно. 

Возможно, стоит разбить эту программу на более маленькие части - например, создавать эти словари постепенно. 